In [ ]:
%load_ext autoreload
%autoreload 2

# Segmentation

## Règles

_Avant de s’interroger sur les responsabilités d’un algorithme, il faut être en mesure d’identifier quel algorithme est la source de telle décision. Ainsi, en parallèle des conférences, la société civile est invitée dans le cadre d’un Hackathon, à contribuer à la résolution de ce problème ouvert via des techniques d’intelligence artificielle._
_Le Hackathon propose un cas simplifié, mais réaliste, de plateforme de livraison de plats cuisinés. La plateforme affiche à l’utilisateur un tarif unique pour chaque livraison, depuis un restaurant jusqu’au lieu de collecte. Plusieurs options sont possibles pour l'établissement de ce tarif : le restaurant décide quelle option de livraison et/ou tarification de livraison s’applique à lui parmi différents algorithmes proposés par la plateforme ou la tarification est complètement à la main du restaurant._

_Pour comprendre les responsabilités à l’œuvre, les participants inscrits auront pour mission de détecter à partir du jeu de données fourni quel algorithme fixe quelle tarification. Afin d’aider les participants, le jeu de données contient des informations récupérées sur la plateforme fictive (e.g. le lieu de livraison) comme des données contextuelles (e.g. des informations sur la météo locale). Toute donnée supplémentaire apportée par les candidats est bienvenue._

## Méthodologie

## Objectif visé
Déterminer, pour chaque commande, le mode de tarification utilisé (variable caché) étant donné le tarif affiché à l'utilisateur (qui est notre observable).

### Données utilisées (et sources)
Météo (jdd fourni)
Géolocalisation des points de livraison (jdd fourni)
Géolocalisation des commerces (jdd fourni)
Jour et heure (jdd fourni)

### Spécifications techniques du code
python3.10
sklearn==1.1.0

## Feature engineering

In [ ]:
import pandas as pd
tarifs = pd.read_csv("tarifs.csv")

In [ ]:
tarifs.head()

#### Adding restaurant id.
#### Also addd addresses (?) (seem to be useless. Maybe the addressed are anonymized ?)

In [ ]:
tarifs["id"] = tarifs.apply(lambda row: str(row["start_lat"]) + str(row["start_lon"]), axis=1)

In [ ]:
"""
tarifs["lat"] = tarifs["start_lat"]
tarifs["lon"] = tarifs["start_lon"]
tarifs.to_csv("tarifs_geo.csv")
!curl -X POST -F data=@tarifs_geo.csv https://api-adresse.data.gouv.fr/reverse/csv/ > tarifs_adresses.csv
"""


### Distance

In [ ]:
import numpy as np

In [ ]:
# Création de la colonne "distance au point de livraison"
tarifs["d"] = np.sqrt((tarifs["end_lon"]-tarifs["start_lon"])**2 + (tarifs["end_lat"]-tarifs["start_lat"])**2)

## Day

In [ ]:
d = {"Monday": 1, "Tuesday": 2, "Wednesday": 3, "Thursday": 4, "Friday": 5, "Saturday": 6, "Sunday": 7}
tarifs["day_code"] = tarifs.apply(lambda row: d[row["day_of_week"]], axis=1)
df = tarifs[["id", "rain", "heat", "day_code", "hour", "fee", "d"]]
df.head(10)

In [ ]:
# prise en compte de la météo

## Méthodologie de validation

In [ ]:
from sklearn.model_selection import train_test_split
## à faire plus tard avant de soumettre

## Analyse de l'algorithme

In [ ]:
# Nos idées, à mettre plus tard dans le README.md (à la fin du hackathon)

## Visualisation de données

In [ ]:
#df.fee.value_counts()

In [ ]:
import seaborn as sns
import plotly # https://plotly.com/python/

sns.pairplot(df)#, palette="id") #, hue="col1", palette="col2")
# visualisation 2D à creuser : (distance OU temps OU nb course) x commmission 
# notion éventuelle de bonus pour certaines courses à certains moments ou certains endroits 
# bonus : (regarder la série temporelle de la tarification moyenne), ou la distribution géographique

### Hypothese 0: constant prices

In [ ]:
df[["fee", "id"]].groupby("id").nunique().values

### Modélisation avec des ellipses pour gérer le bruit sur la distance (pour plus tard ?)

In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.mixture import BayesianGaussianMixture
model = BayesianGaussianMixture(n_components=3, covariance_type="tied")
#model.fit(df[["d", "fee"]])
# https://runebook.dev/fr/docs/scikit_learn/modules/mixture#bgmm (traduction de la doc scikit-learn)
# https://scikit-learn.org/dev/modules/mixture.html#mixture
# make_pipeline(StandardScaler(), model)